We start by downloading a specific release of the components because running from master is not a good way to buid "repetable" systems

In [1]:
!wget https://github.com/kubeflow/pipelines/archive/0.2.5.tar.gz

--2020-03-12 05:09:58--  https://github.com/kubeflow/pipelines/archive/0.2.5.tar.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/kubeflow/pipelines/tar.gz/0.2.5 [following]
--2020-03-12 05:09:58--  https://codeload.github.com/kubeflow/pipelines/tar.gz/0.2.5
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘0.2.5.tar.gz.2’

0.2.5.tar.gz.2          [   <=>              ]   4.02M  7.43MB/s    in 0.5s    

2020-03-12 05:09:59 (7.43 MB/s) - ‘0.2.5.tar.gz.2’ saved [4218104]



In [2]:
!tar -xvf 0.2.5.tar.gz

pipelines-0.2.5/
pipelines-0.2.5/.cloudbuild.yaml
pipelines-0.2.5/.dockerignore
pipelines-0.2.5/.gitattributes
pipelines-0.2.5/.github/
pipelines-0.2.5/.github/ISSUE_TEMPLATE/
pipelines-0.2.5/.github/ISSUE_TEMPLATE/BUG_REPORT.md
pipelines-0.2.5/.gitignore
pipelines-0.2.5/.pylintrc
pipelines-0.2.5/.release.cloudbuild.yaml
pipelines-0.2.5/.style.yapf
pipelines-0.2.5/.travis.yml
pipelines-0.2.5/BUILD.bazel
pipelines-0.2.5/CHANGELOG.md
pipelines-0.2.5/CONTRIBUTING.md
pipelines-0.2.5/LICENSE
pipelines-0.2.5/Makefile
pipelines-0.2.5/OWNERS
pipelines-0.2.5/README.md
pipelines-0.2.5/ROADMAP.md
pipelines-0.2.5/VERSION
pipelines-0.2.5/WORKSPACE
pipelines-0.2.5/backend/
pipelines-0.2.5/backend/Dockerfile
pipelines-0.2.5/backend/Dockerfile.bazel
pipelines-0.2.5/backend/Dockerfile.persistenceagent
pipelines-0.2.5/backend/Dockerfile.scheduledworkflow
pipelines-0.2.5/backend/Dockerfile.viewercontroller
pipelines-0.2.5/backend/Dockerfile.visualization
pipelines-0.2.5/backend/OWNERS
pipelines-0.2.5/bac

In [4]:
import kfp

In [70]:
#tag::loadGCSDLComponent[]
gcs_download_component = kfp.components.load_component_from_file(
    "pipelines-0.2.5/components/google-cloud/storage/download/component.yaml")
#end::loadGCSDLComponent[]
#tag::loadTFDVAndFriendsComponents[]
tfx_csv_gen = kfp.components.load_component_from_file(
    "pipelines-0.2.5/components/tfx/ExampleGen/CsvExampleGen/component.yaml")
tfx_statistic_gen = kfp.components.load_component_from_file(
    "pipelines-0.2.5/components/tfx/StatisticsGen/component.yaml")
tfx_schema_gen = kfp.components.load_component_from_file(
    "pipelines-0.2.5/components/tfx/SchemaGen/component.yaml")
tfx_example_validator = kfp.components.load_component_from_file(
    "pipelines-0.2.5/components/tfx/ExampleValidator/component.yaml")
#end::loadTFDVAndFriendsComponents[]

In [29]:
@kfp.dsl.pipeline(
  name='DL',
  description='Sample DL pipeline'
)
def pipeline_with_dl():
    dl_component = gcs_download_component(
        gcs_path="gs://ml-pipeline-playground/tensorflow-tfx-repo/tfx/components/testdata/external/csv") # Your path goes here

In [31]:
kfp.compiler.Compiler().compile(pipeline_with_dl, 'dl_pipeline.zip')

In [32]:
client = kfp.Client()

In [34]:
my_experiment = client.create_experiment(name='dl')
my_run = client.run_pipeline(my_experiment.id, 'dl', 
  'dl_pipeline.zip')

In [78]:
#tag::standaloneTFDVPipeline[]
@kfp.dsl.pipeline(
  name='TFDV',
  description='TF DV Pipeline'
)
def tfdv_pipeline():
    # DL with wget, can use gcs instead as well
    fetch = kfp.dsl.ContainerOp(
      name='download',
      image='busybox',
      command=['sh', '-c'],
      arguments=[
          'sleep 1;'
          'mkdir -p /tmp/data;'
          'wget https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/trx_data.csv -O /tmp/data/results.csv'],
      file_outputs={'downloaded': '/tmp/data'})
    # This expects a directory of inputs not just a single file
    records_example = tfx_csv_gen(input_base=fetch.output)
    stats = tfx_statistic_gen(input_data=records_example.output)
    schema_op = tfx_schema_gen(stats.output)
    tfx_example_validator(stats=stats.outputs['output'], schema=schema_op.outputs['output'])
#end::standaloneTFDVPipeline[]

In [79]:
kfp.compiler.Compiler().compile(tfdv_pipeline, 'tfdv_pipeline.zip')

In [80]:
my_experiment = client.create_experiment(name='tfdv_pipeline')
my_run = client.run_pipeline(my_experiment.id, 'tfdv', 
  'tfdv_pipeline.zip')